In [24]:
#Chargement des données
# Data
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import math
# Graphics
import seaborn as sns ; sns.set()

from sklearn.experimental import enable_halving_search_cv # noqa

from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
# from sklearn.metrics import root_mean_squared_error

In [25]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_filtre= """
SET search_path to principal;
SELECT *
from "filmview"
where 'Comedy' = ANY(string_to_array("genres", ','))
limit 10000;
"""
SQL= """
SET search_path to principal;
SELECT *
from "filmview"
where "runtimeMinutes" Is NOT null and "titleType" = 'movie' and "averageRating" is NOT NULL
limit 10000;
"""
df = pd.read_sql(SQL, engine)
engine.dispose()
df

,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt0029201,The Man Who Could Work Miracles,movie,0,1936,None,82,"Comedy,Family,Fantasy",6.9,1574,"[actor_Roland_Young, writer_Lajos_Biró, writer..."
1,tt0029202,The Man Who Cried Wolf,movie,0,1937,None,68,"Crime,Drama",6.3,68,"[actor_Forrester_Harvey, actor_Tom_Brown, writ..."
2,tt0029205,The Man in Blue,movie,0,1937,None,67,"Action,Crime,Drama",4.6,17,"[actor_Richard_Carle, actor_Edward_Ellis, acto..."
3,tt0029206,Man of the People,movie,0,1937,None,81,"Crime,Drama,Film-Noir",5.9,122,"[director_Edwin_L._Marin, actor_Thomas_Mitchel..."
4,tt0029207,The Mandarin Mystery,movie,0,1936,None,66,"Comedy,Crime,Film-Noir",5.3,447,"[actress_Charlotte_Henry, writer_Rex_Taylor, w..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,tt0044020,Sensation in San Remo,movie,0,1951,None,98,"Comedy,Musical,Romance",5.8,29,"[composer_Theo_Nordhaus, actress_Elisabeth_Mar..."
9996,tt0044023,The Seven Dwarfs to the Rescue,movie,0,1951,None,84,"Comedy,Family,Fantasy",5.5,57,"[actor_Mario_Mastrantonio, writer_W._Pietrini,..."
9997,tt0044024,Sette ore di guai,movie,0,1951,None,84,Comedy,6.3,119,"[writer_Eduardo_Scarpetta, writer_Agenore_Incr..."
9998,tt0044025,The Seven Deadly Sins,movie,0,1952,None,148,Drama,6.4,267,"[director_Yves_Allégret, director_Carlo_Rim, a..."


In [26]:
def listTostr (df):
    return df.apply(lambda x: ' '.join(map(str, x)))

In [27]:
# df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
# df["startYear"].fillna(df["startYear"].mean(), inplace=True)
# df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)
# df["isAdult"] = df["isAdult"].apply(lambda x: 1 if x == "True" else 0)

# df['isAdult'] = df['isAdult'].astype(bool)
df['Cate&names'].fillna('missing', inplace=True)
df['Cate&names']=df[['Cate&names']].apply(listTostr)
# df['Cate&names']=df['Cate&names'].str.replace('_', ' ')
df['genres'].fillna('missing', inplace=True)
df['genres']=df['genres'].str.replace(',', ' ')
df


,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt0029201,The Man Who Could Work Miracles,movie,0,1936,None,82,Comedy Family Fantasy,6.9,1574,actor_Roland_Young writer_Lajos_Biró writer_H....
1,tt0029202,The Man Who Cried Wolf,movie,0,1937,None,68,Crime Drama,6.3,68,actor_Forrester_Harvey actor_Tom_Brown writer_...
2,tt0029205,The Man in Blue,movie,0,1937,None,67,Action Crime Drama,4.6,17,actor_Richard_Carle actor_Edward_Ellis actor_R...
3,tt0029206,Man of the People,movie,0,1937,None,81,Crime Drama Film-Noir,5.9,122,director_Edwin_L._Marin actor_Thomas_Mitchell ...
4,tt0029207,The Mandarin Mystery,movie,0,1936,None,66,Comedy Crime Film-Noir,5.3,447,actress_Charlotte_Henry writer_Rex_Taylor writ...
...,...,...,...,...,...,...,...,...,...,...,...
9995,tt0044020,Sensation in San Remo,movie,0,1951,None,98,Comedy Musical Romance,5.8,29,composer_Theo_Nordhaus actress_Elisabeth_Marku...
9996,tt0044023,The Seven Dwarfs to the Rescue,movie,0,1951,None,84,Comedy Family Fantasy,5.5,57,actor_Mario_Mastrantonio writer_W._Pietrini ed...
9997,tt0044024,Sette ore di guai,movie,0,1951,None,84,Comedy,6.3,119,writer_Eduardo_Scarpetta writer_Agenore_Incroc...
9998,tt0044025,The Seven Deadly Sins,movie,0,1952,None,148,Drama,6.4,267,director_Yves_Allégret director_Carlo_Rim acto...


In [28]:
# Séparation des caractéristiques et de la cible
X = df.drop(columns=["averageRating"])
y = df["averageRating"]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [29]:
# Définition des colonnes numériques, textuelles et de description
numeric_features = ['startYear', 'runtimeMinutes']
boolean_features = 'isAdult'
text_features = ['titleType']
title = 'primaryTitle'#['primaryTitle''titleType', 'directors', 'writers']
genre = 'genres'
description_feature = 'Cate&names'

In [30]:
# Création des transformers pour les colonnes numériques, booléennes, textuelles et de description
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

boolean_transformer = FunctionTransformer(lambda x: x.astype(bool).values.reshape(-1, 1)) 

text_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Vect_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('vect', CountVectorizer(decode_error='ignore', analyzer='char_wb')) #max_features=1000, analyzer="word"
])
tfidf_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('tf_idf', TfidfVectorizer(decode_error='ignore', analyzer='char_wb')) #max_features=1000
])
# Création d'un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bool', boolean_transformer, boolean_features),
        ('text', text_transformer, text_features),
        ('title', tfidf_transformer, title),
        ('genre', tfidf_transformer, genre),
        ('description', tfidf_transformer, description_feature)
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['startYear', 'runtimeMinutes']),
                                ('bool',
                                 FunctionTransformer(func=<function <lambda> at 0x000001DE4DA8DF80>),
                                 'isAdult'),
                                ('text',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneH...
                                 ['titleType']),
                                ('title',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  decode_error='ignore'))]),
                                 'primaryTitle'),
                                ('genre',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  decode_error='ignore'))]),
                                 'genres'),
                                ('description',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  decode_error='ignore'))]),
                                 'Cate&names')])

In [31]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('preprocessor', preprocessor), ('regressor', ElasticNet())]),
    'Random Forest Regression': Pipeline([('preprocessor', preprocessor), ('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor())]),
    'SVR': Pipeline([('preprocessor', preprocessor), ('regressor', SVR())]),
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {'regressor__fit_intercept': [True,False]},
    'Ridge Regression': {'regressor__alpha': [0.1, 1.0, 5.0]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1.0, 5.0]},
    'ElasticNet': {'regressor__alpha': [0.1, 1.0, 5.0], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'Random Forest Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50,100]},# None, 
    'Gradient Boosting Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50, 100]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
}

# Scoring : RMSE, R2 et MAE
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'R2': 'r2',
           'MAE': 'neg_mean_absolute_error'}


In [32]:
def Grid(X_train, y_train, pipeline, parameters, cv=5):
  # Scoring
  #multi_scoring = {mean_squared_error,r2_score}
    
    
  # Grid search
  grid = HalvingGridSearchCV(pipeline, parameters,  scoring='neg_root_mean_squared_error', refit='RMSE', cv=cv, n_jobs =-1, verbose = 1, error_score='raise')

  # Fit
  grid.fit(X_train, y_train)

  # Scores and results
  best_score = grid.best_score_.round(4)
  best_params = grid.best_params_
  training_time = grid.cv_results_['mean_fit_time'].mean().round(4)

  # Output
  return({
      'best_score': best_score,
      'best_params': best_params,
      'training_time': training_time,
      'fitted_model': grid.best_estimator_
  })

In [33]:
def afficheResults (grid):
    model_name = grid['fitted_model'].named_steps['regressor'].__class__.__name__
    print(f"{model_name} training time: {grid['training_time']}")
    print(f"Best {model_name} parameters: {grid['best_params']}")
    print(f"Best {model_name} score: {-grid['best_score']}")
 
    
    

In [34]:

# Boucle sur les modèles pour ajuster avec GridSearchCV
models = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"\n..............{model_name}..............................")
    grid_search = Grid(X_train, y_train, pipeline, parameters[model_name], cv=5)
    afficheResults (grid_search)
    
    best_model = grid_search['fitted_model']
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    Rmse = round(math.sqrt(mse), 4)
    models[model_name] = [best_model,Rmse]
    print(f"{model_name} RMSE: {Rmse}")
 


..............Linear Regression..............................
n_iterations: 1
n_required_iterations: 1
n_possible_iterations: 1
min_resources_: 8000
max_resources_: 8000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 2
n_resources: 8000
Fitting 5 folds for each of 2 candidates, totalling 10 fits
LinearRegression training time: 2.7018
Best LinearRegression parameters: {'regressor__fit_intercept': True}
Best LinearRegression score: 0.7516
Linear Regression RMSE: 0.744

..............Ridge Regression..............................
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 2666
max_resources_: 8000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 3
n_resources: 2666
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 1
n_candidates: 1
n_resources: 7998
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Ridge training time: 1.5212
Best Ridge parameters: {'regressor__alp

In [ ]:
'''
# Boucle sur les modèles pour ajuster avec GridSearchCV
results = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit='RMSE', cv=5, n_jobs =-1, verbose = 1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = root_mean_squared_error(y_test, y_pred)
    results[model_name] = mse
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best {model_name} score: {-grid_search.best_score_}")
    print(f"{model_name} MSE: {mse}")
    print()
'''

'\n# Boucle sur les modèles pour ajuster avec GridSearchCV\nresults = {}\n\n# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation\nfor model_name, pipeline in pipelines.items():\n    print(f"Training {model_name}...")\n    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit=\'RMSE\', cv=5, n_jobs =-1, verbose = 1)\n    grid_search.fit(X_train, y_train)\n    best_model = grid_search.best_estimator_\n    y_pred = best_model.predict(X_test)\n    mse = root_mean_squared_error(y_test, y_pred)\n    results[model_name] = mse\n    print(f"Best parameters: {grid_search.best_params_}")\n    print(f"Best {model_name} score: {-grid_search.best_score_}")\n    print(f"{model_name} MSE: {mse}")\n    print()\n'

In [ ]:
'''
# Evaluation des modèles sur les données de test
print("\nComparaison des performances des modèles sur les données de test:")
for model_name, mse in results.items():
    print(f"{model_name}: MSE = {mse}")
    
'''

'\n# Evaluation des modèles sur les données de test\nprint("\nComparaison des performances des modèles sur les données de test:")\nfor model_name, mse in results.items():\n    print(f"{model_name}: MSE = {mse}")\n    \n'